# EdgeLock Secure Enclave (nxpele utility)

EdgeLock Secure Enclave (ELE) is NXP's hardware-based security subsystem designed to provide robust protection for sensitive operations and data in embedded systems. It serves as a dedicated security processor that handles cryptographic operations, secure key storage, authentication, and trusted execution environments, effectively isolating critical security functions from the main application processor.

## Nxpele tool

Nxpele is a utility for communicating with the ELE on target devices over blhost (mboot) or U-Boot. See `nxpele --help` for details. 
It provides three communication options with ELE, specified using the `--device (-d)` parameter:

1. **Blhost (mboot)**
   - For i.MX RT1180 MCU, you can communicate with the EdgeLock Enclave via the blhost (mboot) interface.

2. **U-Boot serial**
   - For MPUs like i.MX 8x or i.MX 9x series, you can communicate with EdgeLock Enclave using the U-Boot serial interface. Once U-Boot is booted, find its serial port to establish a connection and send commands to the ELE subsystem.

3. **U-Boot fastboot**
   - An alternative method for communicating with ELE on MPU platforms with U-Boot is through the fastboot protocol. In this case, U-Boot also needs to be running, but communication happens through the fastboot interface, which can provide faster and more reliable data transfer compared to serial connection. Fastboot must be started before communication. You can start fastboot directly in the U-Boot console using the `fastboot` command, or by sending an `nxpele` command with the appropriate U-Boot console COM port and `-d uboot_fastboot` parameter, for example:

   ```bash
   nxpele --port COM3 --device uboot_fastboot --family mimx9596 get-info
   ```

   > **NOTE**: To use the nxpele app over fastboot, you need to build U-Boot with the flag `CONFIG_CONSOLE_MUX=y`.


## Nxpele commands

Nxpele provides several commands to interact with the EdgeLock Secure Enclave (ELE). Below are examples demonstrating common usage scenarios. 
These examples are used on i.MX95 devices, but you can specify your own family to run these commands.

In [1]:
# This env variable sets colored logger output to STDOUT
# Execute this cell to enable execution of the ! line magic
%env JUPYTER_SPSDK=1
%alias execute echo %l && %l
%alias_magic ! execute

FAMILY = "mimx9596"
PORT = "com266"

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


### U-Boot serial interface
Nxpele provides some commands to get information about currently connected device, for example `get-info`, `get-ele-fw-status`, `get-events`, etc.

In [2]:
%! nxpele --family $FAMILY --port $PORT --device uboot_serial get-info

nxpele --family mimx9596 --port com266 --device uboot_serial get-info 
ELE get info ends successfully:
Command:              0xda
Version:              3
Length:               256
SoC ID:               MX95 - 0x9500
SoC version:          B000
Life Cycle:           OEM_CLSD - 0x0040
SSSM state:           4
Attest API version:   0
UUID:                 c9d88ddf5ace444da168dd2a520630e1
SHA256 ROM PATCH:     72d02b666a524aca0a3162accb4c8de7af33083fc2faefb249d87c7de1437c81
SHA256 FW:            9fdfd92f6f1a6e222362b4a9b5110a964df304a66f91e343d25c3be6a88b75c9
Advanced information:
  OEM SRKH:           41a3ab5e956ff6649e843f2686ada51840415d2fa2b6508960098d5bf6f41f27eafeff21a88f3c4b2ccf08895c1440f4b2675fb0bfe5675333b5fcbb3fba9a45
  CSAL state:         EdgeLock secure enclave random context initialization succeed - 0x02
  TRNG state:         TRNG entropy is valid and ready to be read - 0x03
  OEM PQC SRKH:       643dcc626197897115d7cce1ec079c479399af59cd46b9b45b6578fabe86d20e788683151239d8128c

Other important functionality of nxpele is writing/reading fuses using `read-common-fuse` and `write-fuse` commands. 

In [3]:
%! nxpele --family $FAMILY --port $PORT --device uboot_serial read-common-fuse --index 128

nxpele --family mimx9596 --port com266 --device uboot_serial read-common-fuse --index 128 
Read common fuse ends successfully.
Fuse ID_128: 0x5EABA341



In [4]:
INDEX = ""  # specify index of the fuse to write
DATA = ""  # specify data to write to the fuse

```bash
%! nxpele --family $FAMILY --port $PORT --device uboot_serial write-fuse --index $INDEX --data $DATA
```

> **NOTE**: Writing fuses is one-time operation and cannot be reversed. Be very careful when programming fuses.

Other important functionality of nxpele is generating keyblob used for encryption of the images. For more infomation about encrypted boot please refer to [i.MX 95 Encrypted Boot](../ahab/imx95/imx95_encrypted_signed_ahab_uboot.ipynb) documentation.

In [5]:
DEK_KEY = "../_data/keys/dek_key.txt"

%! nxpele --family $FAMILY --port $PORT generate-keyblob DEK --algorithm AES_CBC --key-id 0 --key $DEK_KEY --key-size 128

nxpele --family mimx9596 --port com266 generate-keyblob DEK --algorithm AES_CBC --key-id 0 --key ../_data/keys/dek_key.txt --key-size 128 
ELE generate DEK key blob ends successfully:
0048008101100300dd05d479565080a1859f828a959d6b5f2d94620c1bf9ea2b0287da42c9b46bb51eb302e7b541370dcbbe6d904531e33c29d4df2bb42cd8c4f96b703f6274a33b


### U-Boot fastboot interface

All the functionalities shown before can also be performed using U-Boot's fastboot interface. As mentioned before, fastboot needs to be started directly in the U-Boot console using the `fastboot` command or with the `-d` parameter on the nxpele command:

In [6]:
%! nxpele --family $FAMILY --port $PORT --device uboot_fastboot get-info

nxpele --family mimx9596 --port com266 --device uboot_fastboot get-info 
ELE get info ends successfully:
Command:              0xda
Version:              3
Length:               256
SoC ID:               MX95 - 0x9500
SoC version:          B000
Life Cycle:           OEM_CLSD - 0x0040
SSSM state:           4
Attest API version:   0
UUID:                 c9d88ddf5ace444da168dd2a520630e1
SHA256 ROM PATCH:     72d02b666a524aca0a3162accb4c8de7af33083fc2faefb249d87c7de1437c81
SHA256 FW:            9fdfd92f6f1a6e222362b4a9b5110a964df304a66f91e343d25c3be6a88b75c9
Advanced information:
  OEM SRKH:           41a3ab5e956ff6649e843f2686ada51840415d2fa2b6508960098d5bf6f41f27eafeff21a88f3c4b2ccf08895c1440f4b2675fb0bfe5675333b5fcbb3fba9a45
  CSAL state:         EdgeLock secure enclave random context initialization succeed - 0x02
  TRNG state:         TRNG entropy is valid and ready to be read - 0x03
  OEM PQC SRKH:       643dcc626197897115d7cce1ec079c479399af59cd46b9b45b6578fabe86d20e788683151239d812

When fastboot is started, the command syntax becomes simpler, because fastboot is the default interface when the `-d` parameter is not specified, so you can communicate with ELE just by defining the family of the target device:

In [7]:
%! nxpele --family $FAMILY get-events

nxpele --family mimx9596 get-events 
ELE get events ends successfully.
Event count:     0
